### Prerequsite - run FinalTestPCR.ipynb first

In [5]:
import joblib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Install xlrd package
%pip install xlrd

# all_df = pd.read_excel('TestDatasetExample.xlsx', index_col=False)
all_df = pd.read_csv('TestDatasetExample_2.csv', index_col=False)
IDs = all_df['ID']
all_df.drop('ID', axis=1, inplace=True)
all_df.head()



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,56.881588,0,0,0,1,3,3,999,0,2,...,0.194591,0.194591,2.846439,0.001281,4.168474e+06,131.044541,0.002335,0.109755,0.013383,0.002051
1,60.000000,0,0,1,0,2,1,1,0,3,...,0.309999,0.309996,2.975317,0.007253,1.736585e+05,23.967478,0.011285,0.055890,0.003163,0.009553
2,58.234086,0,0,0,1,3,3,1,1,4,...,0.328377,0.328377,3.785966,0.003185,3.607821e+06,223.279556,0.001334,0.101628,0.010844,0.001194
3,56.881588,0,0,0,1,3,3,999,0,2,...,0.194591,0.194591,2.846439,0.001281,4.168474e+06,131.044541,0.002335,0.109755,0.013383,0.002051
4,60.000000,0,0,1,0,2,1,1,0,3,...,0.309999,0.309996,2.975317,0.007253,1.736585e+05,23.967478,0.011285,0.055890,0.003163,0.009553


# Part 2: RFS Regression

### Pre-process for Regression

In [6]:

from sklearn.impute import SimpleImputer
import pickle

# load gene clf feature names
import json
keep_feat_names = []
with open('gene_clf_selected_features.json', 'rb') as f:
  keep_feat_names = json.load(f)

if 999 not in all_df['Gene'].values:
  print("no missing Gene - skipping gene impute")
else:
  # replace missing gene with classification result
  # see train_gene_classifier.ipynb for more details
  with open('log_reg_gene_classifier.pkl', 'rb') as f:
    log_res_clf = pickle.load(f)

    # rebuild prediction df
    gene_impute_df = all_df.copy()

    y = gene_impute_df['Gene']

    print("before impute:") 
    print(gene_impute_df['Gene'])

    keep_df = gene_impute_df[keep_feat_names]
    replace_index = keep_df[keep_df['Gene'] == 999].index

    # get prediction on missing gene
    target = gene_impute_df.loc[replace_index, keep_feat_names]
    target.drop('Gene', axis=1, inplace=True)

    print("target shape:", target.shape)

    pred = log_res_clf.predict(target)
    gene_impute_df.loc[replace_index, 'Gene'] = pred

    print("after impute:") 
    print(gene_impute_df['Gene'])

    # assign back to all_df
    all_df['Gene'] = gene_impute_df['Gene']

  # Replace missing values with median of the column
  imputer = SimpleImputer(strategy="median", missing_values=999)
  all_df[:] = imputer.fit_transform(all_df)

print(all_df['Gene'])

all_df.head()

before impute:
0       0
1     999
2       1
3       0
4     999
5       1
6       0
7     999
8       1
9       0
10    999
11      1
Name: Gene, dtype: int64
target shape: (4, 14)
after impute:
0     0
1     0
2     1
3     0
4     0
5     1
6     0
7     0
8     1
9     0
10    0
11    1
Name: Gene, dtype: int64
0     0
1     0
2     1
3     0
4     0
5     1
6     0
7     0
8     1
9     0
10    0
11    1
Name: Gene, dtype: int64


/Users/stephen/Library/Caches/pypoetry/virtualenvs/labs-env-NHQiDXzZ-py3.12/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,56.881588,0,0,0,1,3,3,1,0,2,...,0.194591,0.194591,2.846439,0.001281,4.168474e+06,131.044541,0.002335,0.109755,0.013383,0.002051
1,60.000000,0,0,1,0,2,1,1,0,3,...,0.309999,0.309996,2.975317,0.007253,1.736585e+05,23.967478,0.011285,0.055890,0.003163,0.009553
2,58.234086,0,0,0,1,3,3,1,1,4,...,0.328377,0.328377,3.785966,0.003185,3.607821e+06,223.279556,0.001334,0.101628,0.010844,0.001194
3,56.881588,0,0,0,1,3,3,1,0,2,...,0.194591,0.194591,2.846439,0.001281,4.168474e+06,131.044541,0.002335,0.109755,0.013383,0.002051
4,60.000000,0,0,1,0,2,1,1,0,3,...,0.309999,0.309996,2.975317,0.007253,1.736585e+05,23.967478,0.011285,0.055890,0.003163,0.009553


In [7]:
from outlier_removal import removeOutliers
removeOutliers(all_df)
X = all_df

/Users/stephen/Nottingham/ML/labs_env/COMP4139_Machine_Learning/Assignment 2/outlier_removal.py:42: RuntimeWarning: invalid value encountered in scalar divide
  if abs(val - mean)/std > threshold:


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
X_tsne_mri = tsne.fit_transform(X.iloc[:,11:])
standard_scaler = StandardScaler()
Xs_non_mri = standard_scaler.fit_transform(X.iloc[:,0:11])
Xs = np.c_[Xs_non_mri, X_tsne_mri]

ValueError: perplexity must be less than n_samples

### Load Model and Predict

In [ ]:
with open('rfs_regression_model.pkl', 'rb') as f:
  rfs_model = pickle.load(f)

  # predict RFS
  rfs_pred = rfs_model.predict(Xs)
  print(rfs_pred)
  
  all_df['RelapseFreeSurvival (outcome)'] = rfs_pred



[54.07158975 54.07158975 54.07158975 54.07158975 54.07158975 54.07158975
 54.07158975]


In [ ]:
### Save to results to csv
rfs_coutcome = all_df['RelapseFreeSurvival (outcome)']

reg_outcome = pd.DataFrame({'ID': IDs,'RelapseFreeSurvival (outcome)': rfs_coutcome,})

reg_outcome.to_csv('results/regression_outcome.csv', index=False)